*UE Learning from User-generated Data, CP MMS, JKU Linz 2025*
# Exercise 4: Evaluation

Evaluating a recommender system using offline metrics is crucial to ensuring its quality before deployment. The choice of evaluation metrics is typically guided by the specific application scenario of the recommendation system. Therefore, it is essential to understand how each metric is calculated and what it measures.

In this exercise we evaluate accuracy of the three different RecSys we already implemented (``TopPop``, ``ItemKNN`` and ``SVD``). The first two tasks are about predictive quality metrics, precisely about ``Precision@K`` and ``Recall@K`` respectively. Afterwards, we take a look into ranking quality metrics, especially into ``DCG`` and ``nDCG``. At the end all three recommender systems are evaluated based on these metrics. 

The implementations for the three recommender systems are provided in a file ``rec.py`` and are imported later in the notebook.

Make sure to rename the notebook according to the convention:

LUD25_ex04_k<font color='red'><Matr. Number\></font>_<font color='red'><Surname-Name\></font>.ipynb

for example:

LUD25_ex04_k000007_Bond_James.ipynb

## Overview

Please consult the lecture slides and the presentation from UE Session 4 for a recap.

|Metric|Range|Selection criteria|Limitation|
|------|-------------------------------|---------|----------|
|Precision|$\geq 0$ and $\leq 1$|The closer to $1$ the better.|Only for hits in recommendations. Rank-agnostic.                                                        |
|Recall|$\geq 0$ and $\leq 1$|The closer to $1$ the better.|Only for hits in the ground truth. Rank-agnostic.                                                          |
|nDCG|$\geq 0$ and $\leq 1$|The closer to $1$ the better.|Does not penalize for irrelevant/missing items in the ranking. For example, the following two recommended lists 1,1,1 and 1,1,1,0 would be considered equally good, even if the latter contains an irrelevant item. |

## Implementation
In this exercise, as before, you are reqired to write a number of functions. Only implemented functions are graded. Insert your implementations into the templates provided. Please don't change the templates even if they are not pretty. Don't forget to test your implementation for correctness and efficiency. **Make sure to try your implementations on toy examples and sanity checks.**

Please **only use libraries already imported in the notebook**.

In [1]:
import pandas as pd
import numpy as np
from typing import Callable

## <font color='red'>TASK 1/3</font>: Predictive Quality Metrics

### Precision@K

Precision@k evaluates *how many items in the recommendation list are relevant* (hit) in the ground-truth data. Precision@K is calculated separately for every user and then averaged across all users. For each user, the precision score is normalized by **k**.

It is defined as:

$Precision@K = \frac{1}{|Users|} \sum_{u \in Users} \frac{|\text{Relevant items}_u \cap \text{Recommended Items}_u(K)|}{K}$


#### Input:
* prediction - (**not** an interaction matrix!) numpy array with recommendations. Row index corresponds to ``user_id``, column index corresponds to the rank of the contained recommended item. Every cell (i,j) contains ``item id`` recommended to the user (i) on the position (j) in the list. For example:

The following predictions ``[[12, 7, 99], [0, 97, 6]]`` mean that the user with ``id==1`` (second row) got recommended item **0** on the top of the list, item **97** on the second place and item **6** on the third place.

* test_interaction_matrix - (plain interaction matrix, the same format as before!) interaction matrix built from interactions held out as a test set, rows - users, columns - items, cells - 0 or 1

* topK - integer - top "how many" to consider for the evaluation. By default top 10 items are to be considered

#### Output:
* average ``Precision@k`` score across all users

In [2]:
def get_pk_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray, predictions of the recommendation algorithm for each user;
    test_interaction_matrix - np.ndarray, test interaction matrix for each user;
    topK - int, topK recommendations should be evaluated;
    
    returns - float, average precision@K score over all users;
    """
    score = None
    
    # TODO: YOUR IMPLEMENTATION.
    gen_sum=0
    for u, inter_u in enumerate(test_interaction_matrix):
        user_sum=0
        relevant_idx = np.where(inter_u == 1)[0]
        for i in relevant_idx:
            if i in predictions[u,:topK]:
                user_sum += 1        
        gen_sum+=user_sum/topK
        
    n_users = predictions.shape[0]
    score = gen_sum/(n_users)
    
    return score

In [3]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1]])

pk_score = get_pk_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(pk_score, 0.25), "precision@K score is incorrect."

### Recall@K

Recall@k evaluates *how many relevant items in the ground-truth data are in the recommendation list*. Recall@K is calculated separately for every user and then averaged across all users. For each user, the recall score is normalized by the total number of ground-truth items.

It is defined as:  

$Precision@K = \frac{1}{|Users|} \sum_{u \in Users} \frac{|\text{Relevant items}_u \cap \text{Recommended Items}_u(K)|}{|\text{Relevant Items}_u|}$

**Follow the "same" input and output defintion as for Precison@K**.

In [4]:
def get_rk_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray, predictions of the recommendation algorithm for each user;
    test_interaction_matrix - np.ndarray, test interaction matrix for each user;
    topK - int, topK recommendations should be evaluated;
    
    returns - float, average recall@K score over all users;
    """
    score = 0.0
    
    # TODO: YOUR IMPLEMENTATION.
    gen_sum=0
    
    for u, inter_u in enumerate(test_interaction_matrix):
        user_sum=0
        user_inter_sum= 0
        relevant_idx = np.where(inter_u == 1)[0]
        
        for i in relevant_idx:
            user_inter_sum+=1     
            if i in predictions[u,:topK]:
                user_sum += 1    
                
        gen_sum+=user_sum/user_inter_sum
        
    n_users = predictions.shape[0]
    score = gen_sum/(n_users)
    
    return score

In [5]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1]])

rk_score = get_rk_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(rk_score, 1), "recall@K score is incorrect."

## Questions

* Assume a case, a user wants to find all good items. What is more important, high recall or high precision?
* Write a one-sentence situation where high precision is more important than high recall.
* How do recall and precision relate at Rth (Precision@R and Recall@R) position in the ranking (where R is the number of relevant items)?

*-- Answer Here --*
* High recall
* We're recommending something to clients who are well trained profesionals in their field, and they would drop us if we didn't recommend mostly relevant items (for recall e.g. our boss would drop us if we didn't recommend most of the something they want to push).
* At R, recall is 1 if all relevant items are recommender, precision is the fraction of relevant items in the topR predictions.

## <font color='red'>TASK 2/3</font>: Ranking Quality Metrics

Implement DCG and nDCG in the corresponding templates.

### DCG Score

DCG@K measures the relevance of ranked items while giving higher importance to items appearing earlier in the ranking. It incorporates a logarithmic discount factor to penalize relevant items appearing lower in the ranking.

nDCG@K is calculated separately for every user and then averaged across all users. It is defined as:  

$DCG@K = \sum^K_{i=1} \frac{relevancy_i}{log_2(i+1)}$

**Follow the "same" input and output defintion as for Precison@K**.

Don't forget, DCG is calculated for every user separately and then the average is returned.

<font color='red'>**Attention!**</font> Use logarithm with base 2 for discounts! Remember that the top1 recommendation shouldn't get discounted! Users without interactions in the test set shouldn't contribute to the score.

In [6]:
def get_dcg_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user.
    test_interaction_matrix - np.ndarray - test interaction matrix for each user.
    
    returns - float - mean dcg score over all user.
    """
    score = None

    # TODO: YOUR IMPLEMENTATION.
    gen_sum=[]
    
    for u, preds_u in enumerate(predictions):
        if np.sum(test_interaction_matrix[u]) == 0:
            continue
            
        user_sum=0
        for i, pred in enumerate(preds_u[:topK]): 
            if test_interaction_matrix[u, pred] ==1:
                user_sum += 1 / np.log2(i + 2)
                    
        gen_sum.append(user_sum)

    score = np.mean(gen_sum)

    return score

In [7]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1]])

dcg_score = get_dcg_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(dcg_score, 1), "1 expected"

## Questions

* Can DCG score be higher than 1? Why?
* Can the average DCG score be higher than 1? Why?

*-- Answer Here --*
* Yes, in the sum we're adding up values that start at 1 and gradually get smaller and smaller (assuming the values exist, ie we have the relevant items), but generally there's no cap.
* Yes, we average over users, who can have scores above 1, as in the previous bulletpoint.

### nDCG Score

nDCG is a metric that evaluates how well the recommender performs in recommending ranked items to users. Therefore both hit of relevant items and correctness in ranking of these items matter to the nDCG evaluation. The total nDCG score is normalized by the total number of users.

nDCG@K is calculated separately for every user and then averaged across all users. It is defined as:  

$nDCG@K = \frac{DCG@K}{iDCG@K}$

**Follow the "same" input and output defintion as for Precison@K**

<font color='red'>**Attention!**</font> Remember that ideal DCG is calculated separetely for each user and depends on the number of tracks held out for them as a Test set! Use logarithm with base 2 for discounts! Remember that the top1 recommendation shouldn't get discounted!

<font color='red'>**Note:**</font> nDCG is calculated for **every user separately** and then the average is returned. You do not necessarily need to use the function you implemented above. Writing nDCG from scatch might be a good idea as well.

In [8]:
def get_ndcg_score(predictions: np.ndarray, test_interaction_matrix: np.ndarray, topK=10) -> float:
    """
    predictions - np.ndarray - predictions of the recommendation algorithm for each user;
    test_interaction_matrix - np.ndarray, test interaction matrix for each user;
    topK - int, topK recommendations should be evaluated;
    
    returns - float, average ndcg score over all users;
    """   
    score = 0
    # TODO: YOUR IMPLEMENTATION.

    gen_sum=[]

    for u, preds_u in enumerate(predictions):
        if np.sum(test_interaction_matrix[u]) == 0:
            continue
            
        user_sum=0
        for i, pred in enumerate(preds_u[:topK]): 
            if test_interaction_matrix[u, pred] ==1:
                user_sum += 1 / np.log2(i + 2)

        idcg = int(np.sum(test_interaction_matrix[u]))
        idcg = idcg if idcg<topK else topK
            
        idcg_sum=0
        for i in range(idcg):
            idcg_sum+=1 / np.log2(i + 2)
            
        gen_sum.append(user_sum/idcg_sum)

    score = np.mean(gen_sum)

    return score

In [9]:
predictions = np.array([[0, 1, 2, 3], [3, 2, 1, 0], [1, 2, 3, 0], [-1, -1, -1, -1]])
test_interaction_matrix = np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0]])

ndcg_score = get_ndcg_score(predictions, test_interaction_matrix, topK=4)

assert np.isclose(ndcg_score, 1), "ndcg score is not correct."

### Questions

* Can nDCG score be higher than 1?

*-- Answer Here --*
* No, best case scenario we get DCG = iDCG, then we just get n=nr samples in the sum (add 1 for each sample), divide by n to average, we get 1. In suboptimal scenarios the proportion would be less than 1, so we'd get less than n in the average, i.e. (n-x)/n < 1

## <font color='red'>TASK 3/3</font>: Evaluation
Use the provided ``rec.py`` (see imports below) to build a simple evaluation framework. It should be able to evaluate ``POP``, ``ItemKNN`` and ``SVD``.


In [10]:
from rec import inter_matr_implicit
from rec import svd_decompose, svd_recommend_to_list  #SVD
from rec import recTopK  #ItemKNN
from rec import recTopKPop  #TopPop

In [11]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')


users = read("lfm-tiny-tunes", 'user')
items = read("lfm-tiny-tunes", 'item')
train_inters = read("lfm-tiny-tunes", 'inter_train')
test_inters = read("lfm-tiny-tunes", 'inter_test')

train_interaction_matrix = inter_matr_implicit(users=users, items=items, interactions=train_inters,
                                               dataset_name="lfm-tiny-tunes")
test_interaction_matrix = inter_matr_implicit(users=users, items=items, interactions=test_inters,
                                              dataset_name="lfm-tiny-tunes")

### Get Recommendations

In [12]:
def get_recommendations_for_algorithms(config: dict) -> dict:
    """
    config - dict - configuration as defined in the next cell

    returns - dict - already predefined below with name "rec_dict"
    """

    #use this structure to return results
    rec_dict = {"recommenders": {
        "SVD": {
            #Add your predictions here
            "recommendations": np.array([], dtype=np.int64)
        },
        "ItemKNN": {
            "recommendations": np.array([], dtype=np.int64)
        },
        "TopPop": {
            "recommendations": np.array([], dtype=np.int64)
        },
    }}

    # TODO: YOUR IMPLEMENTATION.
    train_inter = config["train_inter"]
    top_k = config["top_k"]
    n_factors = config["recommenders"]["SVD"]["n_factors"]
    n_neighbours = config["recommenders"]["ItemKNN"]["n_neighbours"]
    
    U, V = svd_decompose(train_inter, n_factors)

    n_users = train_inter.shape[0]

    svd_recs = []
    knn_recs = []
    toppop_recs = []
    for u in range(n_users):
        seen = np.where(train_inter[u] > 0)[0].tolist()
        _svd_recs = svd_recommend_to_list(u, seen, U, V, top_k)
        svd_recs.append(_svd_recs)
    
        _knn_recs = recTopK(train_inter, user=u, top_k=top_k, n=n_neighbours)
        knn_recs.append(_knn_recs)
    
        _toppop_recs = recTopKPop(train_inter, user=u, top_k=top_k)
        toppop_recs.append(_toppop_recs)

    rec_dict = {"recommenders": {
        "SVD": {
            #Add your predictions here
            "recommendations": np.array(svd_recs, dtype=np.int64)
        },
        "ItemKNN": {
            "recommendations": np.array(knn_recs, dtype=np.int64)
        },
        "TopPop": {
            "recommendations": np.array(toppop_recs, dtype=np.int64)
        },
    }}

    return rec_dict

In [13]:
config_predict = {
    #interaction matrix
    "train_inter": train_interaction_matrix,
    #topK parameter used for all algorithms
    "top_k": 10,
    #specific parameters for all algorithms
    "recommenders": {
        "SVD": {
            "n_factors": 50
        },
        "ItemKNN": {
            "n_neighbours": 5
        },
        "TopPop": {
        }
    }
}

In [ ]:
recommendations = get_recommendations_for_algorithms(config_predict)

assert "SVD" in recommendations["recommenders"] and "recommendations" in recommendations["recommenders"]["SVD"]
assert isinstance(recommendations["recommenders"]["SVD"]["recommendations"], np.ndarray)
assert np.issubdtype(recommendations["recommenders"]["SVD"]["recommendations"].dtype, np.integer), "Predictions must contain integer indices"
assert "ItemKNN" in recommendations["recommenders"] and "recommendations" in recommendations["recommenders"]["ItemKNN"]
assert isinstance(recommendations["recommenders"]["ItemKNN"]["recommendations"], np.ndarray)
assert np.issubdtype(recommendations["recommenders"]["ItemKNN"]["recommendations"].dtype, np.integer), "Predictions must contain integer indices"
assert "TopPop" in recommendations["recommenders"] and "recommendations" in recommendations["recommenders"]["TopPop"]
assert isinstance(recommendations["recommenders"]["TopPop"]["recommendations"], np.ndarray)
assert np.issubdtype(recommendations["recommenders"]["TopPop"]["recommendations"].dtype, np.integer), "Predictions must contain integer indices"


### Evaluate Recommendations

Implement the function such that it evaluates the previously generated recommendations. Make sure you use the provided config dictionary and pay attention to the structure for the output dictionary.

In [ ]:
def evaluate_algorithms(config: dict, calculate_ndcg_score: Callable,
    calculate_pk_score: Callable, calculate_rk_score: Callable,) -> dict:
    """
    config - dict, configuration containing recommenders and test interaction matrix;
    calculate_ndcg_score - callable, function to calculate the ndcg score;
    calculate_pk_score - callable, function to calculate the precision@k score;
    calculate_rk_score - callable, function to calculate the recall@k score;

    returns - dict, { Recommender Key from input dict: 
        {"ndcg" : float "ndcg score"}
        {"pk" : float "precision@k score"}
        {"rk" : float "recall@k score"}
    };
    """

    metrics = {
        "SVD": {
            "pk": None,
            "rk":None,
            "ndcg": None,
        },
        "ItemKNN": {
            "pk": None,
            "rk":None,
            "ndcg": None,
        },
        "TopPop": {
            "pk": None,
            "rk":None,
            "ndcg": None,
        },
    }

    # TODO: YOUR IMPLEMENTATION.
    test_inter = config["test_inter"]
    top_k = config["top_k"]

    svd_recs = config["recommenders"]["SVD"]["recommendations"]
    knn_recs = config["recommenders"]["ItemKNN"]["recommendations"]
    toppop_recs = config["recommenders"]["TopPop"]["recommendations"]
    
    svd_pk = calculate_pk_score(svd_recs, test_inter, top_k)
    knn_pk = calculate_pk_score(knn_recs, test_inter, top_k)
    toppop_pk = calculate_pk_score(toppop_recs, test_inter, top_k)

    svd_rk = calculate_rk_score(svd_recs, test_inter, top_k)
    knn_rk = calculate_rk_score(knn_recs, test_inter, top_k)
    toppop_rk = calculate_rk_score(toppop_recs, test_inter, top_k)

    svd_ndcg = calculate_ndcg_score(svd_recs, test_inter, top_k)
    knn_ndcg = calculate_ndcg_score(knn_recs, test_inter, top_k)
    toppop_ndcg = calculate_ndcg_score(toppop_recs, test_inter, top_k)

    metrics = {
        "SVD": {
            "pk": svd_pk,
            "rk":svd_rk,
            "ndcg": svd_ndcg,
        },
        "ItemKNN": {
            "pk": knn_pk,
            "rk":knn_rk,
            "ndcg": knn_ndcg,
        },
        "TopPop": {
            "pk": toppop_pk,
            "rk":toppop_rk,
            "ndcg": toppop_ndcg,
        },
    }

    
    return metrics

In [ ]:
config_test = {
    "top_k": 10,
    "test_inter": test_interaction_matrix,
    "recommenders": {}  # here you can access the recommendations from get_recommendations_for_algorithms

}
# add dictionary with recommendations to config dictionary
config_test.update(recommendations)

### Evaluating Every Algorithm
Make sure everything works.
We expect KNN to outperform other algorithms on our small data sample.

In [ ]:
evaluations = evaluate_algorithms(config_test, get_ndcg_score, get_pk_score, get_rk_score)

evaluation_metrics = ["pk", "rk", "ndcg"]
recommendation_algs = ["SVD", "ItemKNN", "TopPop"]

for metric in evaluation_metrics:
    for algorithm in recommendation_algs:
        assert algorithm in evaluations and metric in evaluations[algorithm] and isinstance(evaluations[algorithm][metric], float)

In [ ]:
for recommender in evaluations.keys():
    print(f"{recommender}:")
    print(f"p@k: {evaluations[recommender]['pk']}")
    print(f"r@k: {evaluations[recommender]['rk']}")
    print(f"ndcg: {evaluations[recommender]['ndcg']}\n")

## Questions and Potential Future Work
* How would you try improve performance of all three algorithms?
* What other metrics would you consider to compare these recommender systems?

In [ ]:
# The end.